# Morris Method for Auto-Ranking of Sensitive LE3 Inputs
## Inputs: 
- Morris Trajectories File
- Experiment Results Table File
## Outputs:
- Ranking File
    - Each row contains: OutputName, first 6 sensitive input indices
- Info File
    - Each row contains: OutputName, names of first 6 sensitive inputs

## Current Morris Experiment Intended Ranges:
- Wind Direction `[0, 15]`
- Wind Speed `[3, 20]`
- Air Density `[.95*1.225, 1.05*1.225]`
- Erosion Levels `[0, 1]`
    - Regions 1-2 `*.25`
    - Regions 3-4 `*.5`
    - Regions 5-6 `*1`

In [1]:
from SALib.analyze.morris import analyze
import pandas as pd
import numpy as np


problem = {
    'num_vars': 21,
    'names': ['wd_dir', 'wd_spd','air_dens',
              'b1er1','b1er2','b1er3','b1er4','b1er5','b1er6',
             'b2er1','b2er2','b2er3','b2er4','b2er5','b2er6',
             'b3er1','b3er2','b3er3','b3er4','b3er5','b3er6'],
    'bounds': [[0,1],
               [0,1],
               [0,1],
               [0,1],
               [0,1],
               [0,1],
               [0,1],
               [0,1],
               [0,1],
               [0,1],
               [0,1],
               [0,1],
               [0,1],
               [0,1],
               [0,1],
               [0,1],
               [0,1],
               [0,1],
               [0,1],
               [0,1],
               [0,1],]
}

inputs = np.loadtxt("Morris21_5T.txt", float)

# Load in the corresponding Morris results file
morris_exp_data = pd.read_table("Morris215tmean.txt",delimiter = ',')

# Check the Data

In [2]:
morris_exp_data.info()
allnames = morris_exp_data.columns.tolist()
iter = -1;
for i in allnames:
    iter = iter + 1
    print(iter, i)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110 entries, 0 to 109
Columns: 158 entries, Time to GenTq
dtypes: float64(157), int64(1)
memory usage: 135.9 KB
0 Time
1 WindHubAngXY
2 WindHubVelMag
3 WindHubVelX
4 BldPitch1
5 BldPitch2
6 BldPitch3
7 YawPzn
8 LSShftFxa
9 LSShftFya
10 LSShftFza
11 LSShftMxa
12 LSSTipMya
13 LSSTipMza
14 LSSGagMya
15 LSSGagMza
16 HSShftTq
17 HSShftPwr
18 LSSTipVxa
19 LSSGagVxa
20 RootFxb1
21 RootFxb2
22 RootFxb3
23 RootFyb1
24 RootFyb2
25 RootFyb3
26 RootFzc1
27 RootFzc2
28 RootFzc3
29 RootMxb1
30 RootMxb2
31 RootMxb3
32 RootMyb1
33 RootMyb2
34 RootMyb3
35 RootMzc1
36 RootMzc2
37 RootMzc3
38 TipDxc1
39 TipDxc2
40 TipDxc3
41 TipDyc1
42 TipDyc2
43 TipDyc3
44 TipDzc1
45 TipDzc2
46 TipDzc3
47 TipDxb1
48 TipDxb2
49 TipDxb3
50 TipDyb1
51 TipDyb2
52 TipDyb3
53 TipALxb1
54 TipALxb2
55 TipALxb3
56 TipALyb1
57 TipALyb2
58 TipALyb3
59 TipALzb1
60 TipALzb2
61 TipALzb3
62 TipRDxb1
63 TipRDxb2
64 TipRDxb3
65 TipRDyb1
66 TipRDyb2
67 TipRDyb3
68 HSShftV
69 RotPwr
70 Yaw

In [9]:
# Specify names for the files:
ind_fileId = "Morris21_5T_index.txt"
name_fileId = "Morris21_5T_named.txt"
# Enter the for loop
iter = 0
# Create a lists to hold all the file results
ind_list = []
name_list = []
iter = 1
for i in allnames[4:]:
    
    # Add the Output name to the current lists
    ind_line = i
    name_line = i
    
    # Select the outputs
    outputs = morris_exp_data[[i]].to_numpy()
    
    # Run the sensitivity analysis
    Si = analyze(problem, inputs, outputs, conf_level=0.95,
                    print_to_console=False, num_levels=4)
    # Create an array to hold mu_star and sigma 
    stor = np.zeros((2,21))
    stor[0,:] = Si["mu_star"]
    stor[1,:] = Si["sigma"]
    # Make an array of values to rank the input parameters
    vals = np.zeros((21,))
    for i in range(21):
        vals[i] = np.linalg.norm(stor[:,i])
    # Make a copy
    tempvals = vals
    # List off the top 5 influential inputs...
    for j in range(6):
        ind = np.argmax(tempvals)
        ind_line = ind_line+", "+str(ind)
        # Extract the name
        nm = (Si["names"][ind])
        name_line = name_line+", "+nm
        tempvals[ind] = 0
    ind_line = ind_line + "\n"
    name_line = name_line + "\n"
    ind_list = ind_list + [ind_line]
    name_list = name_list + [name_line]
# Write the results to a file to be used by the LE3_GaSP trainer
indFile = open(ind_fileId, "w")
indFile.writelines(ind_list)
indFile.close()
nameFile = open(name_fileId, "w")
nameFile.writelines(name_list)
nameFile.close()
